# Exercício 1

## Enunciado do Problema

Use a package **Criptography** para 

1. Criar um comunicação privada assíncrona entre um agente ***Emitter*** e um agente ***Receiver*** que cubra os seguintes aspectos:
    1. Autenticação do criptograma e dos metadados (associated data). Usar uma cifra simétrica num modo **HMAC** que seja seguro contra ataques aos “nounces” .
    2. Os “nounces” são gerados por um gerador pseudo aleatório (PRG) construído por um função de hash em modo XOF.
    3. O par de chaves **cipher_key**, **mac_key** , para cifra e autenticação, é acordado entre agentes usando o protocolo ECDH com autenticação dos agentes usando assinaturas ECDSA.

## Descrição do Problema

Precisamos de garantir que uma comunicação entre um emitter(pessoa que envia) e um receiver(pessoa que recebe) ocorra de forma segura e privada. Para tal temos que definir os seguintes aspetos:

1. Autenticar o criptograma e os seus metadados através de uma cifra simétrica segura contra ataques aos "nounces".

- Gerar os "nounces" através de um PRG constituido por uma função de hash em modo XOF.

- Os dois agentes chegarem a um acordo quanto às chaves **cipher_key** e **mac_key** usando o protocolo **ECDH** com a autenticação usando assinaturas ECDSA.

## Abordagem

1. Para autenticar o criptograma, vamos utilizar a função HMAC.
2. Para garantir que a comunicação ocorra de forma privada precisamos de encriptar a mensagem e para isso vamos utilizar uma cifra simétrica.
3. Para garantir que a cifra simétrica é segura contra ataques aos nounces ou *replay attacks* podemos utilizar a função de hash para autenticar o nounce.
4. Para garantir aleatoriedade na geração dos "nounces", vamos utilizar uma função de hash em modo XOF (Extendable Output Function).
5. Para definirmos as chaves **cipher_key** e **mac_key** entre os dois agentes, vamos utilizar o protocolo **ECDH**(Elliptic-curve Diffie–Hellman) .
6. Para autenticar os agentes, vamos utilizar o algoritmo **ECDSA**(Elliptic Curve Digital Signature Algorithm).

## Código de resolução

### Emitter

Esta função demonstra a comunicação num ponto de vista mais "high-level" entre o emitter e o receiver.

In [ ]:
import asyncio
import os

from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

from TP1.ex1.emitter import get_connection, authenticate_and_encrypt_message
from TP1.ex1.encryption import generate_random_nonce, pad_message
from ex1.receiver import RECEIVER_HOST, RECEIVER_PORT, READER_BUFFER_SIZE


async def main():
    reader, writer = await get_connection(RECEIVER_HOST, RECEIVER_PORT)

    while True:
        message = input("Message: ")
        message_bytes = message.encode("utf-8")

        if message in ["q", "quit", "exit"]:
            writer.close()
            break

        cipher_key, mac_key = await initialize_session_emitter(reader, writer)
        print("PRIVATE INFORMATION", "\n\tCipher Key:", cipher_key, "\n\tMAC Key:", mac_key)

        nonce = generate_random_nonce()

        # print("Nonce:", nonce[:10], '...', nonce[-10:])
        print("Nonce:", nonce)

        writer.write(nonce)
        await writer.drain()
        print("Nonce sent.")

        ciphertext, tag, _nonce = authenticate_and_encrypt_message(message_bytes, cipher_key, mac_key, nonce)

        # Send encrypted message and authentication tag to the receiver
        writer.write(tag)
        writer.write(ciphertext)

        print('Tag:', tag[:10], '...', tag[-10:])
        print('Ciphertext:', ciphertext[:10], '...', ciphertext[-10:])
        print("\tTag and ciphertext sent.")

        ack = await reader.read(READER_BUFFER_SIZE)
        assert ack == b"ACK"


Segue-se a função que trata de todos os passos para um comunicação segura entre o emitter e o receiver.
Esta função retorna as chaves **cipher_key** e **mac_key** que são usadas para encriptar e autenticar a mensagem.
Para isto, esta primeiro assina a sua ECDH public key com a sua ECDSA private key e envia a assinatura e a sua ECDH public key para o receiver. De notar que o receiver tem a chave pública ECDSA do emitter e o emitter tem a chave pública ECDH do receiver.
Depois, é criada uma chave partilhada entre o emitter e o receiver através do protocolo ECDH. A partir desta chave partilhada, é criada uma chave de cifra e uma chave de autenticação que são usadas para encriptar e autenticar a mensagem.

De notar, que caso uma assinatura não seja válida, o agente termina o programa, avisando o utilizador de um possível ataque Man-in-the-middle.
Também, caso o autenticador do criptograma não seja válido, o agente termina o programa, avisando o utilizador de um possível ataque de replay.

Do outro lado(do receiver), o processo é analogo.

In [ ]:

from cryptography.hazmat.primitives._serialization import Encoding, PublicFormat
from TP1.ex1.encryption import get_ECDSA_keys_emitter


async def initialize_session_emitter(reader, writer):
    ecdsa_private_key, ecdsa_public_key = await get_ECDSA_keys_emitter()

    ecdh_private_key = ec.generate_private_key(ec.SECP384R1())
    ecdh_public_key = ecdh_private_key.public_key()
    print("\tECDH public key:", ecdh_public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo))

    # Sign ECDH public key with ECDSA private key
    signature = ecdsa_private_key.sign(
        await get_public_bytes(ecdh_private_key),
        ec.ECDSA(hashes.SHA256())
    )

    # Send ECDH public key with signature to the receiver
    writer.write(signature)
    await writer.drain()

    writer.write(await get_public_bytes(ecdh_private_key))
    await writer.drain()
    print("Emitter's public key sent.")

    # Receive receiver's ECDH public key
    signature = await reader.read(104)
    receiver_public_key_bytes = await reader.read(1000)
    print("Receiver's public key received.")
    print("Signature:", signature)
    print("Public key:", receiver_public_key_bytes)

    try:
        receiver_ECDSA_public_key = await get_ECDSA_RECEIVER_public_key()

        receiver_ECDSA_public_key.verify(signature, receiver_public_key_bytes, ec.ECDSA(hashes.SHA256()))
        print("Receiver's signature verified.")
    except InvalidSignature:
        print("Receiver's signature verification failed.")
        raise Exception("Man in the middle attack detected!")

    receiver_public_key_bytes = load_pem_public_key(receiver_public_key_bytes)
    print("\tECDH public key loaded:",
          receiver_public_key_bytes.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo)[:50], "...")

    # Generate shared secret from ECDH key exchange
    shared_secret = ecdh_private_key.exchange(ec.ECDH(), receiver_public_key_bytes)
    print("Shared Secret Derived:", shared_secret[:50], "...")
    print("Shared secret derived.")

    # Derive cipher and MAC keys from shared secret using HKDF
    hkdf = HKDF(algorithm=hashes.SHA256(), length=64, salt=None, info=b'secret')
    hkdf_output = hkdf.derive(shared_secret)
    cipher_key, mac_key = hkdf_output[:32], hkdf_output[32:]

    return cipher_key, mac_key


### Receiver

De um modo semelhante ao emitter, o receiver recebe a assinatura e a chave pública ECDH do emitter e verifica a assinatura. E depois, o receiver envia a sua chave pública ECDH assinada com a sua chave privada ECDSA para o emitter.

In [1]:
async def main():
    server = await asyncio.start_server(connection_handler, RECEIVER_HOST, RECEIVER_PORT)

    addrs = ', '.join(str(sock.getsockname()) for sock in server.sockets)
    print(f'Serving on {addrs}')

    async with server:
        await server.serve_forever()
        server.close()

In [ ]:
async def connection_handler(reader: asyncio.StreamReader, writer: asyncio.StreamWriter):
    while True:
        print("-" * 40, "CONNECTION INFO", "-" * 40)
        try:
            cipher_key, mac_key = await initialize_session_receiver(reader, writer)
        except ValueError:
            print("Emitter closed connection.")
            break

        print("-" * 40, "PRIVATE INFORMATION", "-" * 40, "\nCipher Key:", cipher_key, "\nMAC Key:", mac_key)

        # Receive nonce from the emitter
        print("-" * 40, "MESSAGE INFO", "-" * 40)
        nonce = await reader.read(16)
        print("Nonce:", nonce[:10], '...', nonce[-10:])

        tag = await reader.read(32)
        ciphertext = await reader.read(1000)

        print('Tag:', tag[:10], '...', tag[-10:])
        print('Ciphertext:', ciphertext[:10], '...', ciphertext[-10:])
        print("Tag and ciphertext received.")
        print("-" * 80)

        # Authenticate message with HMAC-SHA256
        try:
            verify_message(ciphertext, key=mac_key, nonce=nonce, tag=tag)
            print("Message authenticated successfully.")
        except InvalidSignature:
            print("!!! MESSAGE AUTHENTICATION FAILED !!!")

        # Decrypt message with AES-256 in CBC mode
        plaintext = decrypt_message(ciphertext, cipher_key, nonce)
        utf8_plaintext = plaintext.decode("utf-8")
        print(f"Plaintext received: \"{utf8_plaintext}\"")

        # Send ACK to emitter
        writer.write(b'ACK')


In [ ]:

async def initialize_session_receiver(reader: asyncio.StreamReader, writer: asyncio.StreamWriter):
    print("Connection from:", writer.get_extra_info("peername"))

    ecdsa_private_key, ecdsa_public_key = await get_ECDSA_keys_receiver()
    ecdh_private_key, ecdh_public_key = await get_ECDH_keys()

    print("ECDSA private key:", ecdsa_private_key.private_bytes(Encoding.PEM, format=PrivateFormat.PKCS8,
                                                                encryption_algorithm=NoEncryption()))
    print("ECDSA public key:", ecdsa_public_key.public_bytes(Encoding.PEM, format=PublicFormat.SubjectPublicKeyInfo))

    # Receive emitter's ECDH public key
    signature = await reader.read(1000)
    emitter_public_key_bytes = await reader.read(1000)
    print("Emitter's public key received.")

    try:
        emitter_ECDSA_public_key = await get_ECDSA_EMITTER_public_key()

        emitter_ECDSA_public_key.verify(signature, emitter_public_key_bytes, ec.ECDSA(hashes.SHA256()))
        print("Emitter's signature verified.")
    except InvalidSignature:
        print("Emitter's signature verification failed.")
        raise Exception("Man in the middle attack detected!")

    # Sign ECDH public key with ECDSA private key
    signature = ecdsa_private_key.sign(
        await get_public_bytes(ecdh_private_key),
        ec.ECDSA(hashes.SHA256())
    )
    print("Signature:", signature)
    print("Signature length:", len(signature))
    print("ECDH public key:", await get_public_bytes(ecdh_private_key))

    # Send ECDH public key with signature to the emitter
    writer.write(signature)
    await writer.drain()

    writer.write(await get_public_bytes(ecdh_private_key))
    await writer.drain()
    print("Receiver's public key sent.")

    # Load emitter's public key
    emitter_public_key = load_pem_public_key(emitter_public_key_bytes)

    # Generate shared secret from ECDH key exchange
    shared_key = ecdh_private_key.exchange(ec.ECDH(), emitter_public_key)
    print("Shared secret derived.")

    # Derive cipher and MAC keys from shared secret using HKDF
    hkdf = HKDF(algorithm=hashes.SHA256(), length=64, salt=None, info=b'secret')
    hkdf_output = hkdf.derive(shared_key)
    print("HKDF output derived.")

    # Split HKDF output into cipher and MAC keys
    cipher_key = hkdf_output[:32]
    mac_key = hkdf_output[32:]

    return cipher_key, mac_key


### Funções de cifra e autenticação

Função que autentica a mensagem usando o algoritmo HMAC256.

In [ ]:
def authenticate_message(message: bytes, key: bytes, nonce: bytes) -> bytes:
    hmac_algorithm = hmac.HMAC(key, hashes.SHA256())
    hmac_algorithm.update(nonce + message)

    tag = hmac_algorithm.finalize()

    return tag

Função que encripta a mensagem usando o algoritmo AES256 com modo OFB(Output FeedBack).

In [ ]:

def encrypt_message(message: bytes, key: bytes, nonce: bytes):
    padded_message = pad_message(message)
    print("Padded message:", padded_message)

    cipher = Cipher(algorithms.AES256(key), modes.OFB(nonce[:16]))
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_message) + encryptor.finalize()

    return ciphertext


Função que verifica a autenticidade da mensagem usando o algoritmo HMAC256.

In [ ]:
def verify_message(message: bytes, key: bytes, nonce: bytes, tag: bytes) -> bool:
    hmac_algorithm = hmac.HMAC(key, hashes.SHA256())
    hmac_algorithm.update(nonce + message)

    hmac_algorithm.verify(tag)

    return True

Função que desencripta a mensagem usando o algoritmo AES256 com modo OFB(Output FeedBack).

In [ ]:

def decrypt_message(ciphertext: bytes, key: bytes, nonce: bytes):
    print("\t\tnonce:", nonce)
    cipher = Cipher(algorithms.AES256(key), modes.OFB(nonce[:16]))
    decryptor = cipher.decryptor()
    unpadded_message = decryptor.update(ciphertext) + decryptor.finalize()

    message = unpad_message(unpadded_message)

    return message

Função geradora de nonces. Esta função usa o algoritmo de hash XOF **shake128** para gerar um nonce pseudo-aleatório.

In [ ]:
NONCE_HASH_FUNCTION = hashes.SHAKE128
NONCE_HASH_SIZE = 16
DIGEST_SIZE = 128


def generate_random_nonce():
    """ Generate random nonce using XOF hash function """
    xof = hashes.Hash(NONCE_HASH_FUNCTION(DIGEST_SIZE))
    xof.update(os.urandom(NONCE_HASH_SIZE))

    return xof.finalize()[0:NONCE_HASH_SIZE]


## Exemplos e testes da aplicação

A aplicação foi desenhada para ser iterativa no sentido de um utilizador no programa emitter poder mandar as mensagens que quiser para o programa receiver. Para tal, o utilizador no programa emitter deve escrever a mensagem que pretende enviar e pressionar a tecla enter. O programa receiver irá receber a mensagem e imprimir a mesma no terminal. Assim o receiver recebe as mensagens enviadas pelo emitter de forma segura e autenticada. E o emitter manda as mensagens também de forma segura e autenticada.